REQUIRED DEPENDENCIES

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
%pip install  \
    "langchain>=0.0.350" \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client==2.2.4 \
    apache-beam==2.52. \
    tiktoken==0.5.2 \
    "ipywidgets>=7,<8" \
    matplotlib==3.8.2 \
    anthropic==0.9.0

In [ ]:
%pip install pydantic==1.10.8

RESTART THE KERNEL

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

IMPORT BEDROCK CLIENT

Establishing a connection to the Berock service using AWS credentials and configuration settings provided through environment variables. This allows the Python script to interact with the Bedrock service using the boto3_bedrock object, which serves as a client for making API requests.

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

SET UP THE MODEL

In [ ]:
    from langchain.llms.bedrock import Bedrock

    inference_modifier = {'max_tokens_to_sample':4096, 
                        "temperature":0.5,
                        "top_k":250,
                        "top_p":1,
                        "stop_sequences": ["\n\nHuman"]
                        }

    textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                        client = boto3_bedrock, 
                        model_kwargs = inference_modifier 
                        )

# Prompting engineering
Prompting engineerins is the process of designing prompts in a way that optimizes the performance of large language models (LLMs) for specific tasks.
In this practice we are going to use the following prompting techniques :
- Zero-shot
- Few-shot
- COT

## Zero-shot prompt templates
Zero-shot prompt engineering involves crafting prompts for language models to enable them to perform tasks without explicit training examples or fine-tuning on that specific task, leveraging their initial training.

Large language models (LLMs) excel at various tasks out of the box:
- Text Generation: Generating text in a specific style or genre without fine-tuning on a large corpus of text in that style.
- Text Classification: Classifying text into predefined categories without explicit training examples for each category.
- Question Answering: Answering questions based on general knowledge without task-specific training data.
- Language Translation: Translating text between languages without training data for specific language pairs.
- Summarization: Generating summaries of text documents without task-specific training on summarization datasets.
- Sentiment Analysis: Analyzing the sentiment of text without labeled examples for each sentiment class.

Today's large LLMs, such as GPT-3.5, are finely tuned to follow instructions and are trained on vast amounts of data, making them capable of performing various tasks "zero-shot."

Here is one example of the Zero-shot techniques used for Sentiment Analysis :

In [15]:
prompt = """
Classify the text into neutral, negative or positive. 
Text: I think the vacation is okay.
Sentiment:
"""

response = textgen_llm(prompt)

print_ww(response)

Note that in the prompt above we didn't provide the model with any examples of text alongside their classifications, the LLM already understands "sentiment" -- that's the zero-shot capabilities at work.

### Exercises :
Try to solve the following problem using the Zero-shot techniques :

#### Exercise 1 - Text Summarization:
One of the standard tasks in natural language generation is text summarization. Text summarization can include many different flavors and domains. In fact, one of the most promising applications of language models is the ability to summarize articles and concepts into quick and easy-to-read summaries. Let's try a basic summarization task using prompts.

Let's say you are interested to learn about antibiotics.

Start by requesting an explanation of antibiotics from the model.
Then, ask the model to provide a concise summary of the definition it provided earlier.


In [ ]:
#Get the definition of antibiotics
prompt = """
...
"""

antibiotics_definition = textgen_llm(prompt)

#Summarize the definition of antibiotics
prompt_2 = antibiotics_definition + "..."
summary = textgen_llm(prompt_2)

print_ww(summary)

#### Exercise 2 - Code generation :
One application where LLMs are quite effective is code generation. Copilot is a great example of this. There are a vast number of code-generation tasks you can perform with clever prompts.

Imagine you're conducting an analysis across various departments within the university.
You need to extract the studentsID and the students names of the computer science department.

Provide info about possible database tables the model should use in the query.
Then ask the model to generate a valid MySQL query.

In [ ]:
#Get the definition of antibiotics
prompt = """
...
"""

SQL_query = textgen_llm(prompt)

print_ww(SQL_query)

## Few-shot prompt templates
While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

Giving Claude examples of how you want it to behave (or how you want it not to behave) is extremely effective for:
- Getting the right answer
- Getting the answer in the right format

#### Exercise 1 - Model behaviour
Pretend you're a developer trying to build a "parent bot" that responds to questions from kids. Claude's default response is quite formal and robotic. This is going to break a child's heart.

You could take the time to describe your desired tone, but it's much easier just to give Claude a few examples of ideal responses.

Try to improve the answer of the model using the Few Shot Prompting technique.

In [ ]:
#Non-working solution with Few-shot technique
prompt = """
User: Will Santa bring me presents on Christmas?
"""

response = textgen_llm(prompt)

print_ww(response)

### Exercise 2 - Formatting

In the following formatting example, we could walk Claude step by step through a set of formatting instructions on how to extract names and professions and then format them exactly the way we want, or we could just provide Claude with some correctly-formatted examples and Claude can extrapolate from there.

The format in which we want to extract names and professions is like the following example :

"
Narrative
"

<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

To solve this exercise we need different narratives that highlight the professional contributions of some people. Let's ask to ChatGPT to provide this type of narrative always using the FEW SHOT prompting technique.
Provide at least 2 or 3 example to teach the model the exact format you want to receive in the answer

In [ ]:
#Non-working solution with Few-shot technique
prompt = """
Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town.
"""

response = textgen_llm.invoke(prompt)

print_ww(response)

## COT : Chain of thoughts

Introduced in [Wei et al. (2022)](https://arxiv.org/abs/2201.11903), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

#### Exercise 1 - Sarcasm :
The next exercise present a prompt in which is asked the model to analyze the sentiment of a film review. It's not easy to detect because the user used sarcasm in its review.

Run the model with the given prompt to observe the answer and then try to improve it by using the COT technique.

In [ ]:
#Example with Few-shot prompting
prompt = """
User: Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. 
In totally unrelated news, I have been living under a rock since the year 1900.
"""

response = textgen_llm(prompt)

print_ww(response)

#### Exercise 2 - Dates :

Change the prompt with COT technique and try to make the model answer correctly.


In [ ]:
prompt = """
User: Name a famous movie starring an actor who was born in the year 1956.
"""

response = textgen_llm(prompt)

print_ww(response)

## Zero-shot COT : 

One recent idea that came out more recently is the idea of zero-shot CoT [(Kojima et al. 2022)]() that essentially involves adding "Let's think step by step" to the original prompt. Let's try a simple problem and see how the model performs:

In [ ]:
#prompt with Zero-shot COT
prompt = """
I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1. How many apples did I remain with?
Let's think step by step.
"""

response = textgen_llm(prompt)

print_ww(response)

--------------------------------------------------------------------
                                    SEE BELOW FOR EXERCISES SOLUTIONS
--------------------------------------------------------------------

## SOLUTIONS

### Zero-shot

In [ ]:
# Exercise 1

#Get the definition of antibiotics
prompt = """
"Explain antibiotics\n"
"A:"
"""

antibiotics_definition = textgen_llm(prompt)

#Summarize the definition of antibiotics
prompt_2 = antibiotics_definition + "\nExplain the above text in one sentence :"
summary = textgen_llm(prompt_2)

print_ww(summary)

In [ ]:
#### Exercise 2

#Get the definition of antibiotics
prompt = """
    Table departments, columns = [DepartmentId, DepartmentName]
    Table students, columns = [DepartmentId, StudentId, StudentName]
    Create a MySQL query for all students in the Computer Science Department
    """

SQL_query = textgen_llm(prompt)

print_ww(SQL_query)

### Few-shot


In [ ]:
#### Exercise 1
#Solution with Few-shot technique
prompt = """
User: Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. 
There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?
"""

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#### Exercise 2
#Solution with Few-shot technique
prompt = """
User: In the bustling town of Emerald Hills, a diverse group of individuals made their
mark. Sarah Martinez, a dedicated nurse, was known for her compassionate care
at the local hospital.
David Thompson, an innovative software engineer, worked tirelessly on groundbreaking
projects that would revolutionize the tech industry.
Meanwhile, Emily Nakamura, a talented artist and muralist, painted vibrant and thought-provoking
pieces that adorned the walls of buildings and galleries alike. 
Lastly, Michael O'Connell, an ambitious entrepreneur, opened a unique, eco-friendly cafe that quickly
became the town's favorite meeting spot.
Each of these individuals contributed to the rich tapestry of the Emerald Hills community.
<individuals>
1. Sarah Martinez [NURSE]
2. David Thompson [SOFTWARE ENGINEER]
3. Emily Nakamura [ARTIST]
4. Michael O'Connell [ENTREPRENEUR]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his 
farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients
has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen
has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand
the library's offerings and establish reading programs for children have had a significant 
impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful
murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres,
whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic 
swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train 
the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and 
dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer 
known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating
has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy
of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and
self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives.
Her commitment to improving the lives of others has been instrumental in creating a strong sense
of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven
themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town.
Assistant: <individuals>
"""

response = textgen_llm.invoke(prompt)

print_ww(response)

### COT - Chain of thaugths

In [ ]:
#### Exercise 1

#Example with Few-shot prompting
prompt = """
User: Is this review sentiment positive or negative? 
Reason step by step firstly analyzing the positive side and then the negative one

This movie blew my mind with its freshness and originality. 
In totally unrelated news, I have been living under a rock since 1900.
"""


response = textgen_llm.invoke(prompt)

print_ww(response)

In [ ]:
#Exercise 2
prompt = """
User: Name a famous movie starring an actor who was born in the year 1956. 
First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer.
"""

response = textgen_llm.invoke(prompt)

print_ww(response)